In [ ]:
import difflib
import os
import re

question_answer = """
walk through the night she's my life i love you.
baby you're a rich man from the sun
she's gonna have some fun,
gonna love you
i'm to come and see me your man.
i wanna be your mother should...)
your mother should know (your mother should know (your mother should know (yeah.)
your mother should know (your mother should know (your mother should have lasted to me
you’ve got to the bottom and i see me alone
i'm feeling blue.
and it’s my life i love you,
baby i love you.
i love you, yeah, yeah
yeah yeah yeah.
yes"""
filepath = os.path.join("data", "beatles.txt")

# fragment_separators = [".", ",", "?", "!"]
fragment_separators = [".", "?", "!"]

threshold = 0.75
txt_files = {}


def clean_document(doc):
    l = doc.encode("ascii", "ignore").decode()
    # Remove all punctuation (i.e., quotes, commas, !, ?, etc.)
    l = re.sub(r"[^\w\s]", "", l)
    # Remove all Twitter handle mentions (i.e., “@UTK_EECS“ should be deleted.)
    l = re.sub(r"@[\w]+", " ", l)
    # Remove all instances of double-spaces.
    l = re.sub(r"\s+", " ", l)
    # Remove all 1-2 letter words
    l = re.sub(r"\W*\b\w{1,1}\b", "", l)
    # Convert all characters to lowercase.
    l = l.lower()
    # Remove all whitespace
    l = l.strip()

    return l


def get_closest(prompt, cutoff):
    closest_matches = {}

    for k, v in txt_files.items():
        matches = difflib.get_close_matches(prompt, v, n=5, cutoff=cutoff)
        if len(matches) > 0:
            closest_matches[k] = matches

    if len(closest_matches) == 0:
        return None
    return closest_matches


def read_txt(file_path):
    print("# READING: {}".format(file_path))
    txt_files[file_path] = set()
    with open(file_path, "r") as in_f:
        for doc in in_f.readlines():
            doc = clean_document(doc)
            txt_files[file_path].add(doc)

    print(f"=> {file_path}: {len(txt_files[file_path])} documents")


def get_ratio(str_orig, str_match):
    return difflib.SequenceMatcher(None, str_match, str_orig).ratio()


for sep in fragment_separators:
    question_answer = question_answer.replace(sep, "\n")
a_fragments = question_answer.split("\n")
for i1 in range(len(a_fragments)):
    a_fragments[i1] = clean_document(a_fragments[i1])
a_fragments = list(filter(None, a_fragments))

avg = []
results = {
    "Exact": 0,
    "Partial": 0,
    "Total": 0,
}

read_txt(filepath)

print("=" * 80)

for i, a_frag in enumerate(a_fragments):
    results[a_frag] = {}
    closest_matches = get_closest(a_frag, threshold)
    if closest_matches is None:
        continue
    print(f"{i + 1}. {a_frag}")

    closest_matches_new = {}
    for match, match_list in closest_matches.items():
        mat_list = []
        for match_sentence in match_list:
            ratio = get_ratio(match_sentence, a_frag)
            if ratio == 1.0:
                results["Exact"] += 1
            else:
                results["Partial"] += 1
            avg.append(ratio)
            mat_list.append((match_sentence, ratio))
            print(f"   {round(ratio * 100, 2)}% [{match}] ~ [{match_sentence}]")

        closest_matches_new[match] = mat_list

    results[a_frag].update(closest_matches_new)

results["Total"] = results["Exact"] + results["Partial"]
results["Likelihood"] = sum(avg) / len(avg)
print("=" * 80)
print(f"# TOTAL FRAGMENTS: {results['Total']}")
print(
    f"   => Exact: {results['Exact']} / {results['Total']} "
    f"({round(results['Exact'] / results['Total'] * 100, 2)}%)"
)
print(
    f"   => Partial: {results['Partial']} / {results['Total']} "
    f"({round(results['Partial'] / results['Total'] * 100, 2)}%)"
)
print(
    f"   => Total: {len(avg)} / {results['Total']}"
    + f" ({round(len(avg) / results['Total'] * 100, 2)}%)"
)

In [ ]:
from pprint import pprint

pprint(results)